In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import datetime
#import math
import difflib
import sys, os

sys.path.append('.\\src\\model_util')
from FantasyFootballOffense import calcPastOffensiveFantasyValue

#load up the serialized models
paModelFile = "C:\\Users\\kjree\\Workspaces\\ml-fantasy-football-model\\src\\model_data\\qb_prime_age_model.pkl"
pa_model = pickle.load(open(paModelFile, 'rb'))

#1. Available QB training data has already been divided into 2 sets with a ratio 60/40 train/test
gamesDf = pd.read_json('C:\\Users\\kjree\\Workspaces\\nfl-player-stats\\games_QB_train.json')
testGamesDf = pd.read_json('C:\\Users\\kjree\\Workspaces\\nfl-player-stats\\games_QB_test.json')
gamesDf = gamesDf.append(testGamesDf)

#2. Generate dictionary of all players and the year of their first season in the NFL
firstYearDict = dict()
for player_id in gamesDf.player_id.unique():
    gameList = gamesDf.loc[gamesDf.player_id == player_id]
    if gameList is not None and len(gameList) > 0:
        for index, row in gameList.iterrows():
            seasonYear = 0
            if (row.date.month >=8 and row.date.month <=12):
                seasonYear = row.date.year
            else:
                seasonYear = row.date.year - 1
                
            if firstYearDict.get(player_id) is None:
                firstYearDict[player_id] = int(seasonYear)
            else:
                if int(seasonYear) < firstYearDict.get(player_id):
                    firstYearDict[player_id] = int(seasonYear)


In [5]:
# param: pa_model - prime age regression model that determines a player's prime age
# param: fantasyvalue - fantasy value last year
# param: years_played - years played by this player in the next year
# return: some value that represents the fantasy value next year
def predictOffensiveFantasyValue(paModel, fantasyvalue, years_played):
    paArray = np.array([years_played])
    paValue = paModel.predict(paArray[:,np.newaxis])
    print("Fantasy Value Last Year: ", fantasyvalue)
    print("Prime Age Score: ", (1.5*paValue[0]))
    predictedFantasyValue = fantasyvalue + (1.5*paValue[0])
    #print("Predicted Fantasy Value: ", predictedFantasyValue)
    return predictedFantasyValue

#an alternative to difflib.ratio
def calcDiffScore(predictValues,actualValues):
    
    if len(predictValues) == 0:
        return 1.0
    if len(predictValues) != len(actualValues):
        return 1.0
    
    predictValuesDict = dict()
    actualValuesDict = dict()
    
    pos = 1
    for value in predictValues:
        predictValuesDict[value] = pos
        pos = pos + 1

    pos = 1
    for value in actualValues:
        actualValuesDict[value] = pos
        pos = pos + 1
        
    diffscore = 0
    totalsize = len(predictValues)
    for value in predictValues:
        predictIndex = predictValuesDict[value]
        actualIndex = actualValuesDict[value]
        distance = abs(predictIndex - actualIndex)
        diffscore = diffscore + distance
    
    finalscore = diffscore / totalsize
    return finalscore

In [6]:
#3.  Start with 1978 season for a 40 year study
firstSeason = 1978
lastSeason = None

now = datetime.datetime.now()

if(now.month < 3):
    lastSeason = now.year - 2
else:
    lastSeason = now.year - 1
    
def filterBySeason(date):
    if (date.month >=8 and date.month <=12):
        return date.year
    else:
        return date.year - 1

gamesDf['season'] = gamesDf['date'].apply(filterBySeason)

predStatDict = None
nextStatDict = None
totalDiffScore = 0
totalDiffCount = 0

#4.  Find all players who played in currSeason and calculate their fantasy values for that year
#5.  Take those same players and calculate their actual fantasy values for the next year
for currSeason in range(firstSeason,lastSeason):

    predStatDict = dict()
    nextStatDict = dict()
    print("Testing QB prime age and top performer models for year: ", currSeason)

    #compute the nextSeason
    nextSeason = currSeason + 1

    #get the games for the current season N
    gamesForCurrSeason = gamesDf.loc[gamesDf['season'] == currSeason]
    #get the games for the next season N+1
    gamesForNextSeason = gamesDf.loc[gamesDf['season'] == nextSeason]
    
    #PREDICT THE VALUES
    #loop through the games for current season N
    for player_id in gamesForCurrSeason.player_id.unique():
        gameListNextSeason = gamesForNextSeason.loc[gamesForNextSeason.player_id == player_id]
        gameListCurrSeason = gamesForCurrSeason.loc[gamesForCurrSeason.player_id == player_id]
        if (gameListCurrSeason is not None and len(gameListCurrSeason) > 0) and \
        (gameListNextSeason is not None and len(gameListNextSeason) > 0):
            currSeasonValue = calcPastOffensiveFantasyValue(gameListCurrSeason,currSeason)
            firstYear = firstYearDict.get(player_id)
            years_played = 0
            if firstYear is not None:
                years_played = nextSeason - firstYear + 1
            nextSeasonValuePredict = predictOffensiveFantasyValue(pa_model,currSeasonValue,years_played)
            predStatDict[player_id] = [nextSeasonValuePredict]
    
    #THESE ARE THE ANSWERS!
    #loop through the games for next season N+1
    for player_id in gamesForNextSeason.player_id.unique():
        gameListNextSeason = gamesForNextSeason.loc[gamesForNextSeason.player_id == player_id]
        gameListCurrSeason = gamesForCurrSeason.loc[gamesForCurrSeason.player_id == player_id]
        if (gameListCurrSeason is not None and len(gameListCurrSeason) > 0) and \
        (gameListNextSeason is not None and len(gameListNextSeason) > 0):
            nextSeasonValue = calcPastOffensiveFantasyValue(gameListNextSeason,nextSeason)
            nextStatDict[player_id] = [nextSeasonValue]
    
    #SCORE THE DIFFERENCES
    predictDf = pd.DataFrame.from_dict(predStatDict).transpose()
    predictDf.columns = ['fantasyvalue']
    predictDf.sort_values(by=['fantasyvalue'],ascending=False,inplace=True)
    #print(predictDf)
    nextDf = pd.DataFrame.from_dict(nextStatDict).transpose()
    nextDf.columns = ['fantasyvalue']
    nextDf.sort_values(by=['fantasyvalue'],ascending=False,inplace=True)
    #print(nextDf)

    sm1 = difflib.SequenceMatcher(None,a=predictDf.index.values,b=nextDf.index.values)
    diffScore = sm1.ratio()*100
    #diffScore = calcDiffScore(predictDf.index.values,nextDf.index.values)
    #print(predictDf.index.values)
    #print(nextDf.index.values)
    print("Fantasy Value Diff Score: ", diffScore)
    totalDiffScore = totalDiffScore + diffScore
    totalDiffCount = totalDiffCount + 1
    
print("Average Fantasy Value Diff Score: ", (totalDiffScore / totalDiffCount))

Testing QB prime age and top performer models for year:  1978
Fantasy Value Last Year:  0
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  0
Prime Age Score:  76.74002950685886
Fantasy Value Last Year:  236
Prime Age Score:  79.37168186429992
Fantasy Value Last Year:  78
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  7
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  267
Prime Age Score:  79.52435052801863
Fantasy Value Last Year:  0
Prime Age Score:  79.08055544804637
Fantasy Value Last Year:  60
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  116
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  91
Prime Age Score:  78.68460623888708
Fantasy Value Last Year:  9
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  117
Prime Age Score:  76.74002950685886
Fantasy Value Last Year:  57
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  92
Prime Age Score:  74.56416740582264
Fantasy Value Last Year:  163

Fantasy Value Last Year:  235
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  257
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  112
Prime Age Score:  77.77712396789909
Fantasy Value Last Year:  20
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  165
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  183
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  189
Prime Age Score:  78.68460623888708
Fantasy Value Last Year:  -1
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  0
Prime Age Score:  73.62653940890473
Fantasy Value Diff Score:  20.33898305084746
Testing QB prime age and top performer models for year:  1981
Fantasy Value Last Year:  5
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  4
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  86
Prime Age Score:  78.68460623888708
Fantasy Value Last Year:  7
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  187
Prime Age Score:  7

Fantasy Value Last Year:  289
Prime Age Score:  79.37168186429992
Fantasy Value Last Year:  159
Prime Age Score:  78.68460623888708
Fantasy Value Diff Score:  15.517241379310345
Testing QB prime age and top performer models for year:  1984
Fantasy Value Last Year:  105
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  40
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  9
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  0
Prime Age Score:  77.77712396789909
Fantasy Value Last Year:  49
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  35
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  64
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  92
Prime Age Score:  79.37168186429992
Fantasy Value Last Year:  6
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  32
Prime Age Score:  76.74002950685886
Fantasy Value Last Year:  89
Prime Age Score:  63.72108625597795
Fantasy Value Last Year:  117
Prime Age Score:  

Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  95
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  143
Prime Age Score:  79.08055544804637
Fantasy Value Last Year:  57
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  7
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  115
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  187
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  34
Prime Age Score:  79.52435052801863
Fantasy Value Last Year:  71
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  34
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  134
Prime Age Score:  63.72108625597795
Fantasy Value Last Year:  100
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  11
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  224
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  13
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  367
Prime Age Score:  77.9

Fantasy Value Diff Score:  22.413793103448278
Testing QB prime age and top performer models for year:  1989
Fantasy Value Last Year:  214
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  60
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  0
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  48
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  122
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  131
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  30
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  98
Prime Age Score:  79.52435052801863
Fantasy Value Last Year:  77
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  0
Prime Age Score:  76.74002950685886
Fantasy Value Last Year:  1
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  0
Prime Age Score:  79.52435052801863
Fantasy Value Last Year:  0
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  36
Prime Age Score:  77

...

Fantasy Value Last Year:  63
Prime Age Score:  79.52435052801863
Fantasy Value Diff Score:  14.705882352941178
Testing QB prime age and top performer models for year:  2007
Fantasy Value Last Year:  43
Prime Age Score:  79.08055544804637
Fantasy Value Last Year:  14
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  10
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  26
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  7
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  472
Prime Age Score:  78.68460623888708
Fantasy Value Last Year:  0
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  252
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  119
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  26
Prime Age Score:  76.74002950685886
Fantasy Value Last Year:  5
Prime Age Score:  79.08055544804637
Fantasy Value Last Year:  9
Prime Age Score:  69.80171494267435
Fantasy Value Last Year:  249
Prime Age Score:  7

Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  92
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  372
Prime Age Score:  77.77712396789909
Fantasy Value Diff Score:  16.94915254237288
Testing QB prime age and top performer models for year:  2009
Fantasy Value Last Year:  176
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  147
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  30
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  5
Prime Age Score:  79.08055544804637
Fantasy Value Last Year:  33
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  127
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  45
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  277
Prime Age Score:  79.37168186429992
Fantasy Value Last Year:  26
Prime Age Score:  79.52435052801863
Fantasy Value Last Year:  375
Prime Age Score:  79.52435052801863
Fantasy Value Last Year:  1
Prime Age Score:  48.75758066835007
Fantasy V

Fantasy Value Last Year:  185
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  0
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  3
Prime Age Score:  63.72108625597795
Fantasy Value Last Year:  223
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  96
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  84
Prime Age Score:  76.87429749566284
Fantasy Value Last Year:  171
Prime Age Score:  69.80171494267435
Fantasy Value Last Year:  58
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  0
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  0
Prime Age Score:  79.08055544804637
Fantasy Value Last Year:  40
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  1
Prime Age Score:  78.68460623888708
Fantasy Value Last Year:  131
Prime Age Score:  78.53880230709623
Fantasy Value Last Year:  57
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  78
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  107

Fantasy Value Diff Score:  29.310344827586203
Testing QB prime age and top performer models for year:  2014
Fantasy Value Last Year:  268
Prime Age Score:  77.5167001923055
Fantasy Value Last Year:  134
Prime Age Score:  76.74002950685886
Fantasy Value Last Year:  116
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  228
Prime Age Score:  76.74002950685886
Fantasy Value Last Year:  0
Prime Age Score:  77.92490767220175
Fantasy Value Last Year:  360
Prime Age Score:  56.615202085381696
Fantasy Value Last Year:  29
Prime Age Score:  63.72108625597795
Fantasy Value Last Year:  304
Prime Age Score:  63.72108625597795
Fantasy Value Last Year:  20
Prime Age Score:  79.08055544804637
Fantasy Value Last Year:  140
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  65
Prime Age Score:  79.52435052801863
Fantasy Value Last Year:  175
Prime Age Score:  73.62653940890473
Fantasy Value Last Year:  22
Prime Age Score:  79.37168186429992
Fantasy Value Last Year:  41
Prime Age S

Fantasy Value Diff Score:  13.793103448275861
Average Fantasy Value Diff Score:  18.904214144230732


In [9]:
#6. Make QB predictions for 2018
#FANTASY VALUES FROM 2017
pred2018Dict = dict()
  
#get all 2017 games
gamesForLastSeason = gamesDf.loc[gamesDf['season'] == 2017]
    
#loop through the games for last season
for player_id in gamesForLastSeason.player_id.unique():
    gameList = gamesForLastSeason.loc[gamesForLastSeason.player_id == player_id]
    if gameList is not None and len(gameList) > 0:
        lastSeasonValue = calcPastOffensiveFantasyValue(gameList,2017)
        firstYear = firstYearDict.get(player_id)
        years_played = 0
        if firstYear is not None:
            years_played = nextSeason - firstYear + 1
        print("Player ID: ", player_id)
        nextSeasonValuePredict = predictOffensiveFantasyValue(pa_model,lastSeasonValue,years_played)
        pred2018Dict[player_id] = [nextSeasonValuePredict]
        
pred2018Df = pd.DataFrame.from_dict(pred2018Dict).transpose()
pred2018Df.columns = ['fantasyvalue']
pred2018Df.sort_values(by=['fantasyvalue'],ascending=False,inplace=True)
print(pred2018Df)

#profileDf = pd.read_json('C:\\Users\\kjree\\Workspaces\\nfl-player-stats\\profile_QB_train.json')
#testProfileDf = pd.read_json('C:\\Users\\kjree\\Workspaces\\nfl-player-stats\\profile_QB_test.json')
#profileDf = profileDf.append(testProfileDf)

#for player_id in pred2018Df.index.unique():
#    playerName = profileDf.loc[profileDf.player_id == player_id].name.get_values()
#    print("Name: ", playerName)

Player ID:  19494
Fantasy Value Last Year:  0
Prime Age Score:  61.440671082190576
Player ID:  19561
Fantasy Value Last Year:  208
Prime Age Score:  79.52435052801863
Player ID:  20475
Fantasy Value Last Year:  111
Prime Age Score:  78.53880230709623
Player ID:  2064
Fantasy Value Last Year:  241
Prime Age Score:  79.08055544804637
Player ID:  21331
Fantasy Value Last Year:  270
Prime Age Score:  78.68460623888708
Player ID:  22183
Fantasy Value Last Year:  169
Prime Age Score:  76.74002950685886
Player ID:  2269
Fantasy Value Last Year:  370
Prime Age Score:  40.3714907575679
Player ID:  2335
Fantasy Value Last Year:  0
Prime Age Score:  61.440671082190576
Player ID:  2358
Fantasy Value Last Year:  274
Prime Age Score:  48.75758066835007
Player ID:  23691
Fantasy Value Last Year:  174
Prime Age Score:  61.440671082190576
Player ID:  2410
Fantasy Value Last Year:  -1
Prime Age Score:  79.08055544804637
Player ID:  2447
Fantasy Value Last Year:  158
Prime Age Score:  73.62653940890473
P